## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,georgetown,MY,5.4112,100.3354,81.95,87.0,20.0,2.30,few clouds
1,coquimbo,CL,-29.9533,-71.3436,67.73,68.0,0.0,16.11,clear sky
2,tuatapere,NZ,-46.1333,167.6833,51.82,84.0,77.0,1.77,broken clouds
3,jiayuguan,CN,39.8167,98.3000,20.89,51.0,6.0,1.05,clear sky
4,mataura,NZ,-46.1927,168.8643,51.48,88.0,73.0,2.01,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,georgetown,MY,5.4112,100.3354,81.95,87.0,20.0,2.30,few clouds
5,san pedro,PH,14.3500,121.0167,78.35,82.0,20.0,3.44,few clouds
8,kapaa,US,22.0752,-159.3190,70.12,85.0,100.0,4.61,heavy intensity rain
12,menongue,AO,-14.6585,17.6910,73.13,54.0,55.0,7.27,broken clouds
13,castro,BR,-24.7911,-50.0119,86.99,41.0,0.0,4.79,clear sky
18,east london,ZA,-33.0153,27.9116,76.14,73.0,0.0,18.41,clear sky
23,saint-philippe,RE,-21.3585,55.7679,81.99,82.0,83.0,11.21,broken clouds
25,bowen,AU,-20.0167,148.2333,70.02,76.0,84.0,10.11,broken clouds
26,hithadhoo,MV,-0.6000,73.0833,82.35,71.0,72.0,16.40,broken clouds
27,mar del plata,AR,-38.0023,-57.5575,86.02,50.0,75.0,15.01,broken clouds


In [5]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City                   233
Country                231
Lat                    233
Lng                    233
Max Temp               233
Humidity               233
Cloudiness             233
Wind Speed             233
Current Description    233
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
clean_cities_df = preferred_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()

City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,georgetown,MY,81.95,few clouds,5.4112,100.3354,
5,san pedro,PH,78.35,few clouds,14.3500,121.0167,
8,kapaa,US,70.12,heavy intensity rain,22.0752,-159.3190,
12,menongue,AO,73.13,broken clouds,-14.6585,17.6910,
13,castro,BR,86.99,clear sky,-24.7911,-50.0119,
18,east london,ZA,76.14,clear sky,-33.0153,27.9116,
23,saint-philippe,RE,81.99,broken clouds,-21.3585,55.7679,
25,bowen,AU,70.02,broken clouds,-20.0167,148.2333,
26,hithadhoo,MV,82.35,broken clouds,-0.6000,73.0833,
27,mar del plata,AR,86.02,broken clouds,-38.0023,-57.5575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters
params = {"radius": 5000, "type": "lodging", "key": g_key}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name
nan_value = float("NaN")
hotel_df.replace("",nan_value, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
655,potchefstroom,ZA,70.47,clear sky,-26.7167,27.1000,Melrose Place Guestrooms
656,swellendam,ZA,79.18,few clouds,-34.0226,20.4417,De Kloof Luxury Estate boutique Hotel & SPA
657,bintulu,MY,79.21,scattered clouds,3.1667,113.0333,900 Boutique INN
660,phalaborwa,ZA,78.08,scattered clouds,-23.9430,31.1411,Bothabelo Bed and Breakfast
663,kulhudhuffushi,MV,82.04,broken clouds,6.6221,73.0700,Haajy Guest House 1
673,boca do acre,BR,84.02,overcast clouds,-8.7522,-67.3978,Hotel Sao Pedro
676,mettur,IN,76.86,few clouds,11.8000,77.8000,Hotel Akash Residency
680,deniliquin,AU,74.88,overcast clouds,-35.5333,144.9667,Riviana Motel
681,la palma,US,70.16,clear sky,33.8464,-118.0467,La Quinta Inn & Suites by Wyndham Buena Park
684,isangel,VU,79.90,scattered clouds,-19.5500,169.2667,Tanna Lodge


In [10]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_vacation.csv', index="City", index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating=False, max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))